let us consider a sentence "my name is saugat"

In [1]:
import numpy as np
import math

In [3]:
l,d_k,d_v=4,8,8  #l=length="my name is saugat"=4 words , d_k=8,d_v=8 means size of vector=8



#now we need 3 main things for attention. they are:


1.   q=query=what do i want
2.   k=key=what can i offer
3.   v=value=what i actually offer




In [4]:
#Now lets define q,k,v as random for now
q=np.random.randn(l,d_k)
k=np.random.randn(l,d_k)
v=np.random.randn(l,d_v)


In [6]:
print(f"q={q}")
print(f"k={k}")
print(f"v={v}")

q=[[-1.15873707 -1.12292661  1.53719464  2.69145197 -0.21722089 -0.57057489
   0.39686817  2.57939819]
 [-1.13092477 -0.04624123 -1.21146713  0.22530178 -0.01245056  1.69367177
  -0.79877572 -0.48267566]
 [ 2.23581263 -0.47178513  0.95159789  0.34461021 -1.82811743  0.67997646
   0.05341568  0.97523561]
 [-0.18395556  0.83001059 -2.05269769  1.62774787 -0.22820215 -1.25004423
  -0.05574591  0.73908923]]
k=[[ 0.53702096  2.98720983 -0.08718354  1.20646853 -0.97903543  1.57424501
   1.3398012  -0.92501926]
 [ 1.92510878 -1.09181966 -0.78434258  0.24847455 -0.1529471  -0.28065854
   0.67195458 -0.97122017]
 [-1.03892145  0.78427791 -1.35731363  0.86191042 -0.47354578  0.76130175
  -0.03671953  0.63143657]
 [-0.53205992  1.38977394  0.19709206 -0.02607246 -0.73646591 -0.65656239
   0.55806003 -0.64903416]]
v=[[-0.13599561  0.04135474  2.11439876  0.24253907 -0.39828407 -1.6500081
   0.64308759  0.55259813]
 [ 0.05992497  0.83490384  0.27801984  0.43445172  0.88956349  3.00592478
  -0.72710

#now lets define self attention mechanism:
self_attention=softmax(q.k(transpose)/sqrt(d base k)+M)
new_V=self_attention.V

In [7]:
np.matmul(q,k.T) #k.T=k(transpose)

array([[-3.40337575, -3.58671758,  1.83912343, -1.62934799],
       [ 1.68670443, -1.66187496,  3.99704961, -0.94251342],
       [ 2.1538507 ,  3.33602341, -1.69023543, -1.3699463 ],
       [ 2.02059773,  0.38457712,  4.65633999,  1.28238409]])

so why do we devide by sqrt(d base k)??

*   it is so that we can minimize the variance



In [16]:
#before applying variance
q.var(), k.var(), np.matmul(q,k.T).var()

(1.4370473931251346, 1.016542583159945, 6.118019208434202)

In [17]:
#now lets devide
scaled=np.matmul(q,k.T)/math.sqrt(d_k)
q.var(), k.var(), scaled.var()

(1.4370473931251346, 1.016542583159945, 0.764752401054275)

In [18]:
scaled #notice the reduction on variance of the product

array([[-1.20327504, -1.26809616,  0.65022833, -0.57606151],
       [ 0.59634007, -0.58756153,  1.41317044, -0.33322882],
       [ 0.76150122,  1.17946239, -0.59758847, -0.48434916],
       [ 0.71438918,  0.13596855,  1.64626479,  0.45339124]])

#masking: usually masking is required for decoder part , not for encoder, but lets just define it for now

*   This is to ensure words don't get context from words generated in the future.
    

*   Not required in the encoders, but required int he decoders



In [19]:
mask=np.tril(np.ones((l,l)))
mask

array([[1., 0., 0., 0.],
       [1., 1., 0., 0.],
       [1., 1., 1., 0.],
       [1., 1., 1., 1.]])

In [22]:
mask[mask==0]=-np.infty
mask[mask==1]=1
mask

array([[  1., -inf, -inf, -inf],
       [  1.,   1., -inf, -inf],
       [  1.,   1.,   1., -inf],
       [  1.,   1.,   1.,   1.]])

1 -inf -inf -inf in 1st row means "my"
1 1 1 -inf -inf means "my name"////ans so on

In [23]:
scaled+mask

array([[-0.20327504,        -inf,        -inf,        -inf],
       [ 1.59634007,  0.41243847,        -inf,        -inf],
       [ 1.76150122,  2.17946239,  0.40241153,        -inf],
       [ 1.71438918,  1.13596855,  2.64626479,  1.45339124]])

#now lets add softmax function here
softmax is used to conver the vector into probability distribution so that they can add up to one


In [25]:
def softmax(x):
  return (np.exp(x).T / np.sum(np.exp(x), axis=-1)).T

In [26]:
attention=softmax(scaled+mask)

In [27]:
attention

array([[1.        , 0.        , 0.        , 0.        ],
       [0.7656486 , 0.2343514 , 0.        , 0.        ],
       [0.36026218, 0.54718843, 0.09254938, 0.        ],
       [0.20532476, 0.11514271, 0.52137447, 0.15815807]])

In [28]:
new_v=np.matmul(attention,v)
new_v
#after applying attention

array([[-0.13599561,  0.04135474,  2.11439876,  0.24253907, -0.39828407,
        -1.6500081 ,  0.64308759,  0.55259813],
       [-0.09008135,  0.22732409,  1.68404078,  0.28751407, -0.09647519,
        -0.55888369,  0.32198097,  0.4035691 ],
       [-0.11026173,  0.51105081,  0.83235821,  0.23905174,  0.40785452,
         1.01217123, -0.02359462,  0.2512284 ],
       [-0.57095546,  0.11647654, -0.07169047, -0.3130395 ,  0.24467874,
        -0.08955743,  0.95826876,  0.79797535]])

In [29]:
#before apply attention
v

array([[-0.13599561,  0.04135474,  2.11439876,  0.24253907, -0.39828407,
        -1.6500081 ,  0.64308759,  0.55259813],
       [ 0.05992497,  0.83490384,  0.27801984,  0.43445172,  0.88956349,
         3.00592478, -0.7271053 , -0.08332308],
       [-1.01629969,  0.42466558, -0.88070762, -0.92980485,  0.69781517,
        -0.41275816,  1.54067858,  1.05610224],
       [-0.12683142, -1.32498769, -0.49736559,  0.45469565, -0.88388432,
         0.74812078,  0.67449836,  0.90720994]])

In [30]:
#now lets functionize it
def softmax(x):
  return (np.exp(x).T / np.sum(np.exp(x), axis=-1)).T


def scaled_dot_product_attention(q, k, v, mask=None):
  d_k= q.shape[-1]
  scaled=np.matmul(q,k.T)/math.sqrt(d_k)

  if mask is not None:
    scaled=scaled+mask

  attention=softmax(scaled)
  new_v=np.matmul(attention,v)
  return new_v,attention


In [31]:
values, attention = scaled_dot_product_attention(q, k, v, mask=mask)
print("Q\n", q)
print("K\n", k)
print("V\n", v)
print("New V\n", values)
print("Attention\n", attention)#here we used mask

Q
 [[-1.15873707 -1.12292661  1.53719464  2.69145197 -0.21722089 -0.57057489
   0.39686817  2.57939819]
 [-1.13092477 -0.04624123 -1.21146713  0.22530178 -0.01245056  1.69367177
  -0.79877572 -0.48267566]
 [ 2.23581263 -0.47178513  0.95159789  0.34461021 -1.82811743  0.67997646
   0.05341568  0.97523561]
 [-0.18395556  0.83001059 -2.05269769  1.62774787 -0.22820215 -1.25004423
  -0.05574591  0.73908923]]
K
 [[ 0.53702096  2.98720983 -0.08718354  1.20646853 -0.97903543  1.57424501
   1.3398012  -0.92501926]
 [ 1.92510878 -1.09181966 -0.78434258  0.24847455 -0.1529471  -0.28065854
   0.67195458 -0.97122017]
 [-1.03892145  0.78427791 -1.35731363  0.86191042 -0.47354578  0.76130175
  -0.03671953  0.63143657]
 [-0.53205992  1.38977394  0.19709206 -0.02607246 -0.73646591 -0.65656239
   0.55806003 -0.64903416]]
V
 [[-0.13599561  0.04135474  2.11439876  0.24253907 -0.39828407 -1.6500081
   0.64308759  0.55259813]
 [ 0.05992497  0.83490384  0.27801984  0.43445172  0.88956349  3.00592478
  -0.72

In [32]:
values, attention = scaled_dot_product_attention(q, k, v, mask=None)
print("Q\n", q)
print("K\n", k)
print("V\n", v)
print("New V\n", values)
print("Attention\n", attention)#here we didnt use mask

Q
 [[-1.15873707 -1.12292661  1.53719464  2.69145197 -0.21722089 -0.57057489
   0.39686817  2.57939819]
 [-1.13092477 -0.04624123 -1.21146713  0.22530178 -0.01245056  1.69367177
  -0.79877572 -0.48267566]
 [ 2.23581263 -0.47178513  0.95159789  0.34461021 -1.82811743  0.67997646
   0.05341568  0.97523561]
 [-0.18395556  0.83001059 -2.05269769  1.62774787 -0.22820215 -1.25004423
  -0.05574591  0.73908923]]
K
 [[ 0.53702096  2.98720983 -0.08718354  1.20646853 -0.97903543  1.57424501
   1.3398012  -0.92501926]
 [ 1.92510878 -1.09181966 -0.78434258  0.24847455 -0.1529471  -0.28065854
   0.67195458 -0.97122017]
 [-1.03892145  0.78427791 -1.35731363  0.86191042 -0.47354578  0.76130175
  -0.03671953  0.63143657]
 [-0.53205992  1.38977394  0.19709206 -0.02607246 -0.73646591 -0.65656239
   0.55806003 -0.64903416]]
V
 [[-0.13599561  0.04135474  2.11439876  0.24253907 -0.39828407 -1.6500081
   0.64308759  0.55259813]
 [ 0.05992497  0.83490384  0.27801984  0.43445172  0.88956349  3.00592478
  -0.72